### Merge Shot locations and play by play

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# import files to merge
shot_locations = pd.read_csv("../data/processed/Shot_Locations_top_20_players_2000to2020.csv")
play_by_play = pd.read_csv("../data/processed/PlaybyPlay_2000-2020-v4.csv")


In [2]:
# merge
df_merged = shot_locations.merge(play_by_play, how='outer', left_on=['Game ID','Game Event ID'], right_on=['GAME_ID','EVENTNUM'])

# save as csv
df_merged.to_csv('../data/processed/PlaybyPlay_ShotLocations.csv')

In [3]:
print("Play by play shape:", play_by_play.shape)
print("Play by play games:", play_by_play['GAME_ID'].nunique())
print("Shot locations shape:", shot_locations.shape)
print("Shot locations games:", shot_locations['Game ID'].nunique())
print("Merged shaped:", df_merged.shape)

Play by play shape: (481976, 22)
Play by play games: 13929
Shot locations shape: (334331, 33)
Shot locations games: 13927
Merged shaped: (481976, 55)


In [4]:
# check if observations are missing on the play by play part
len(df_merged[df_merged['GAME_ID'].isna()])

0

In [5]:
# check if observations are missing on the shot location part
len(df_merged[df_merged['Game ID'].isna()])

138565

In [6]:
df_merged[df_merged['Game ID'].isna()].free_throw.value_counts(normalize=True)

free_throw
1.0    0.999459
0.0    0.000541
Name: proportion, dtype: float64

In [7]:
# > 99.5% of missing observations are free throws, we can try to add the shot locations manually

### Merge players stats per year

In [2]:
df_sl_pbp = pd.read_csv("../data/processed/PlaybyPlay_ShotLocations.csv", index_col=0)
df_players = pd.read_csv("../data/processed/stat_joueurs.csv", index_col=0)

In [7]:
# Fill Year feature with data from complete lines 
games = df_sl_pbp[['GAME_ID', 'Year']].drop_duplicates().dropna()

for _, game in games.iterrows():
    df_sl_pbp.loc[df_sl_pbp.GAME_ID==game['GAME_ID'], 'Year']=game['Year']

In [8]:
df_sl_pbp.Year.isna().sum()

4

In [9]:
df_sl_pbp.head(2)

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Shot Distance,X Location,Y Location,Shot Made Flag,Home Team,Away Team,Season Type,Year,Shot Zone Basic_Above the Break 3,Shot Zone Basic_Backcourt,Shot Zone Basic_In The Paint (Non-RA),Shot Zone Basic_Left Corner 3,Shot Zone Basic_Mid-Range,Shot Zone Basic_Restricted Area,Shot Zone Basic_Right Corner 3,Shot Zone Area_Back Court(BC),Shot Zone Area_Center(C),Shot Zone Area_Left Side Center(LC),Shot Zone Area_Left Side(L),Shot Zone Area_Right Side Center(RC),Shot Zone Area_Right Side(R),Shot Zone Range_16-24 ft.,Shot Zone Range_24+ ft.,Shot Zone Range_8-16 ft.,Shot Zone Range_Back Court Shot,Shot Zone Range_Less Than 8 ft.,GAME_ID,EVENTNUM,target,PERIOD,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,OPPONENT_TEAM,at_home,PREVIOUS_OFF_REBOUND,PREVIOUS_DEF_REBOUND,PREVIOUS_OFF_TURNOVER,PREVIOUS_OFF_MISSED,PREVIOUS_EVENTMSGTYPE,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,DETAILLED_SHOT_TYPE,minutes_left,seconds_left
0,20000001.0,11.0,947.0,Allen Iverson,1.610613e+09,Philadelphia 76ers,1.0,19.0,-107.0,167.0,0.0,NYK,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,20000001,11,0,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FREE_THROW,0.0,1.0,0.0,0.0,0.0,0.0,JUMP SHOT,11,668
1,20000001.0,32.0,947.0,Allen Iverson,1.610613e+09,Philadelphia 76ers,1.0,21.0,-115.0,177.0,1.0,NYK,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,20000001,32,1,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FIELD_GOAL_MADE,0.0,1.0,0.0,0.0,0.0,0.0,JUMP SHOT,7,477


In [11]:
# Check if all 20 players are there
df_players.groupby('Player')['Year'].count()

Player
Allen Iverson            11
Chris Webber              9
Dirk Nowitzki            20
Dwight Howard            16
Dwyane Wade              16
Giannis Antetokounmpo     7
James Harden             11
Jason Kidd               14
Jimmy Butler              9
Kawhi Leonard             9
Kevin Durant             12
Kevin Garnett            17
Kobe Bryant              17
LeBron James             17
Paul Pierce              18
Russell Westbrook        12
Shaquille O'Neal         12
Stephen Curry            11
Steve Nash               15
Tim Duncan               17
Name: Year, dtype: int64

In [12]:
df_sl_pbp.shape

(481976, 55)

In [13]:
# merge everything
data = df_sl_pbp.merge(df_players, how='left', left_on=['PLAYER1_NAME', 'Year'], right_on=['Player', 'Year'])

In [14]:
# drop duplicated columns
data = data.drop(['Game ID', 'Game Event ID', 'Player Name', 'Player', 'Team Name', 'Period', 'Player ID', 'Team ID' ], axis = 1)

### Deal with NAs

In [15]:
data.columns

Index(['Shot Distance', 'X Location', 'Y Location', 'Shot Made Flag',
       'Home Team', 'Away Team', 'Season Type', 'Year',
       'Shot Zone Basic_Above the Break 3', 'Shot Zone Basic_Backcourt',
       'Shot Zone Basic_In The Paint (Non-RA)',
       'Shot Zone Basic_Left Corner 3', 'Shot Zone Basic_Mid-Range',
       'Shot Zone Basic_Restricted Area', 'Shot Zone Basic_Right Corner 3',
       'Shot Zone Area_Back Court(BC)', 'Shot Zone Area_Center(C)',
       'Shot Zone Area_Left Side Center(LC)', 'Shot Zone Area_Left Side(L)',
       'Shot Zone Area_Right Side Center(RC)', 'Shot Zone Area_Right Side(R)',
       'Shot Zone Range_16-24 ft.', 'Shot Zone Range_24+ ft.',
       'Shot Zone Range_8-16 ft.', 'Shot Zone Range_Back Court Shot',
       'Shot Zone Range_Less Than 8 ft.', 'GAME_ID', 'EVENTNUM', 'target',
       'PERIOD', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'OPPONENT_TEAM',
       'at_home', 'PREVIOUS_OFF_REBOUND', 'PREVIOUS_DEF_REBOUND',
       'PREVIOUS_OFF_TURNOVER',

In [16]:
data.isna().sum()

Shot Distance                            138565
X Location                               138565
Y Location                               138565
Shot Made Flag                           138565
Home Team                                138565
Away Team                                138565
Season Type                              138565
Year                                          4
Shot Zone Basic_Above the Break 3        138565
Shot Zone Basic_Backcourt                138565
Shot Zone Basic_In The Paint (Non-RA)    138565
Shot Zone Basic_Left Corner 3            138565
Shot Zone Basic_Mid-Range                138565
Shot Zone Basic_Restricted Area          138565
Shot Zone Basic_Right Corner 3           138565
Shot Zone Area_Back Court(BC)            138565
Shot Zone Area_Center(C)                 138565
Shot Zone Area_Left Side Center(LC)      138565
Shot Zone Area_Left Side(L)              138565
Shot Zone Area_Right Side Center(RC)     138565
Shot Zone Area_Right Side(R)            

In [17]:
# update shot location for all free throws 
data.loc[data['free_throw']==1, 'Shot Zone Basic_Mid-Range'] = True
data.loc[data['free_throw']==1, 'Shot Zone Area_Center(C)'] = True
data.loc[data['free_throw']==1, 'Shot Zone Range_8-16 ft.'] = True
data.loc[data['free_throw']==1, 'Shot Distance'] = 15.0
data.loc[data['free_throw']==1, 'X Location'] = 0
data.loc[data['free_throw']==1, 'Y Location'] = 150

In [18]:
data.fillna({'Shot Zone Basic_Above the Break 3':False}, inplace=True)
data.fillna({'Shot Zone Basic_Backcourt': False}, inplace=True)
data.fillna({'Shot Zone Basic_In The Paint (Non-RA)': False}, inplace=True)
data.fillna({'Shot Zone Basic_Left Corner 3' : False}, inplace=True)
data.fillna({'Shot Zone Basic_Mid-Range' : False}, inplace=True)
data.fillna({'Shot Zone Basic_Restricted Area' : False}, inplace=True)
data.fillna({'Shot Zone Basic_Right Corner 3' : False}, inplace=True)
data.fillna({'Shot Zone Area_Back Court(BC)' : False}, inplace=True)
data.fillna({'Shot Zone Area_Center(C)' : False}, inplace=True)
data.fillna({'Shot Zone Area_Left Side Center(LC)' : False}, inplace=True)
data.fillna({'Shot Zone Area_Left Side(L)' : False}, inplace=True)
data.fillna({'Shot Zone Area_Right Side Center(RC)' : False}, inplace=True)
data.fillna({'Shot Zone Area_Right Side(R)' : False}, inplace=True)
data.fillna({'Shot Zone Range_16-24 ft.' : False}, inplace=True)
data.fillna({'Shot Zone Range_24+ ft.' : False}, inplace=True)
data.fillna({'Shot Zone Range_8-16 ft.' : False}, inplace=True)
data.fillna({'Shot Zone Range_Back Court Shot' : False}, inplace=True)
data.fillna({'Shot Zone Range_Less Than 8 ft.' : False}, inplace=True)
data.fillna({"3P%":0}, inplace=True)


C:\Users\Sarah\AppData\Local\Temp\ipykernel_3856\2905369477.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna({'Shot Zone Basic_Above the Break 3':False}, inplace=True)
C:\Users\Sarah\AppData\Local\Temp\ipykernel_3856\2905369477.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna({'Shot Zone Basic_Backcourt': False}, inplace=True)
C:\Users\Sarah\AppData\Local\Temp\ipykernel_3856\2905369477.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call resul

In [19]:
# drop Shot Made Flag : target is the same with no NAs
data.drop('Shot Made Flag', axis=1, inplace = True)

# drop Home Team : PLAYER1_TEAM_ABBREVIATION is the same with no NAs
data.drop('Home Team', axis=1, inplace = True)

In [20]:
# Fill Season Type and Away Team with data from complete lines
games = data[['GAME_ID', 'Away Team', 'Season Type']].drop_duplicates().dropna()

for _, game in games.iterrows():
    data.loc[data.GAME_ID==game['GAME_ID'], 'Away Team']=game['Away Team']
    data.loc[data.GAME_ID==game['GAME_ID'], 'Season Type']=game['Season Type']

In [21]:
# check missing years for some players
data[data.Age.isna()][['Year', 'PLAYER1_NAME']].drop_duplicates().dropna()

,Year,PLAYER1_NAME
159437,2008.0,Chris Webber
200728,2010.0,Allen Iverson
223742,2011.0,Shaquille O'Neal
261703,2013.0,Jason Kidd
287169,2014.0,Steve Nash
322360,2016.0,Kevin Garnett
322408,2016.0,Tim Duncan
322992,2016.0,Kobe Bryant
347118,2017.0,Paul Pierce
377287,2019.0,Dirk Nowitzki


In [22]:
data.describe()

,Shot Distance,X Location,Y Location,Season Type,Year,GAME_ID,EVENTNUM,target,PERIOD,at_home,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,minutes_left,seconds_left,Age,TS%,PTM,ASTM,ORBM,STLM,BLKM,TOVM,USG%,FG%,2P%,3P%,FT%,PTS,year_start,year_end,height,weight,C,PF,PG,PG-SG,SF,SF-SG,SG,SG-PG
count,481901.000000,481901.000000,481901.000000,481972.000000,481972.000000,4.819760e+05,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,481976.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,481976.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000,470495.000000
mean,12.816798,-1.505058,102.242604,0.135300,2009.639369,2.387195e+07,260.629139,0.561833,2.506332,0.519729,0.146594,0.438455,0.159386,0.042523,0.022939,0.287338,4.975771,327.233698,28.399090,0.571727,23.038886,5.200773,1.462233,1.365051,0.891477,2.997611,28.934459,0.482006,0.513064,0.291400,0.784408,1639.148437,2002.088630,2016.258101,198.759269,230.435837,0.141234,0.199734,0.192927,0.004695,0.202534,0.002650,0.249935,0.006291
std,8.209410,85.037931,79.560224,0.342044,5.739282,7.028466e+06,162.014521,0.496162,1.145748,0.499611,0.353702,0.496198,0.366036,0.201779,0.149709,0.452521,3.387594,204.124417,4.329361,0.042106,5.835216,2.445007,0.913823,0.516588,0.682877,0.890281,4.952223,0.051545,0.051553,0.128459,0.107000,551.483158,5.980087,2.699659,10.047883,31.298123,0.348263,0.399801,0.394596,0.068360,0.401888,0.051414,0.432975,0.079068
min,0.000000,-250.000000,-52.000000,0.000000,2000.000000,2.000000e+07,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.000000,0.456000,3.210526,0.400000,0.040000,0.160000,0.048780,0.352941,11.300000,0.358000,0.390667,0.000000,0.417000,35.000000,1993.000000,2008.000000,182.880000,165.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,-13.000000,17.000000,0.000000,2005.000000,2.050094e+07,115.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,153.000000,25.000000,0.545000,18.830986,3.086957,0.772152,0.939394,0.375000,2.400000,25.500000,0.449000,0.482000,0.266000,0.738000,1307.000000,1997.000000,2016.000000,192.024000,212.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,0.000000,134.000000,0.000000,2010.000000,2.110025e+07,261.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,317.000000,28.000000,0.574000,23.953846,4.970588,1.169014,1.378049,0.710145,2.987805,29.600000,0.476000,0.506000,0.327000,0.811000,1735.000000,1999.000000,2018.000000,198.120000,235.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,17.000000,11.000000,150.000000,0.000000,2014.000000,2.170054e+07,384.000000,1.000000,3.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,8.000000,497.000000,31.000000,0.601000,27.200000,6.941176,1.790210,1.759494,1.215686,3.517241,32.200000,0.506000,0.540000,0.374000,0.860000,2036.000000,2008.000000,2018.000000,207.264000,245.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,87.000000,250.000000,867.000000,1.000000,2020.000000,4.990009e+07,872.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,720.000000,40.000000,0.696000,36.128205,11.631579,4.253165,2.800000,2.925926,5.728395,41.700000,0.729000,0.732000,0.600000,1.000000,2832.000000,2014.000000,2018.000000,216.408000,325.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
data.shape

(481976, 72)

In [24]:
data.isna().sum()

Shot Distance                               75
X Location                                  75
Y Location                                  75
Away Team                                    4
Season Type                                  4
Year                                         4
Shot Zone Basic_Above the Break 3            0
Shot Zone Basic_Backcourt                    0
Shot Zone Basic_In The Paint (Non-RA)        0
Shot Zone Basic_Left Corner 3                0
Shot Zone Basic_Mid-Range                    0
Shot Zone Basic_Restricted Area              0
Shot Zone Basic_Right Corner 3               0
Shot Zone Area_Back Court(BC)                0
Shot Zone Area_Center(C)                     0
Shot Zone Area_Left Side Center(LC)          0
Shot Zone Area_Left Side(L)                  0
Shot Zone Area_Right Side Center(RC)         0
Shot Zone Area_Right Side(R)                 0
Shot Zone Range_16-24 ft.                    0
Shot Zone Range_24+ ft.                      0
Shot Zone Ran

## Add defensive rate of opponent team

In [25]:
metrics = pd.read_csv("../data/raw/team_metrics.csv", index_col=0)

# Ajout du defensive rate de l'équipe opposée
data = data.merge(metrics[['ABBREVIATION', 'Year', 'E_DEF_RATING']], left_on=['OPPONENT_TEAM', 'Year'], right_on=['ABBREVIATION', 'Year'])
data.drop('ABBREVIATION', axis=1, inplace=True)

# Ajout de l'offensive rate de l'équipe qui tire
data = data.merge(metrics[['ABBREVIATION', 'Year', 'E_OFF_RATING']], left_on=['PLAYER1_TEAM_ABBREVIATION', 'Year'], right_on=['ABBREVIATION', 'Year'])
data.drop('ABBREVIATION', axis=1, inplace=True)

## Export 

In [26]:
data.dropna(inplace=True)
data.to_csv('../data/processed/all_shots-v6.csv')

## Add pct depending on the previous action and pct depending on the area

In [27]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

shots = pd.read_csv("../data/processed/all_shots-v6.csv", index_col=0)

In [28]:
# import datasets
pct_area = pd.read_csv("../data/processed/pourcentage_par_zone.csv", index_col=0)
pct_action = pd.read_csv("../data/processed/pourcentage_par_action_precedente.csv", index_col=0)
pct_action.dropna(inplace=True)
pct_area.dropna(inplace=True)

# get dummies to match the columns' names of shots
pct_action = pd.concat([pct_action, pd.get_dummies(pct_action.PREVIOUS, prefix="PREVIOUS")], axis=1)
pct_area = pd.concat([pct_area, pd.get_dummies(pct_area["Shot Zone"], prefix="Shot Zone", prefix_sep=" ")], axis=1)

# drop unneeded columns
pct_action.drop(['PREVIOUS','Total_Target',	'Count'], axis=1, inplace=True)
pct_area.drop(["Shot Zone",	"Total_Target",	"Count"], axis=1, inplace=True)

# rename PCT column
pct_action.rename({'Pourcentage':'PCT_PREV_ACTION'}, axis=1, inplace=True)
pct_area.rename({'Pourcentage':'PCT_AREA'}, axis=1, inplace=True)

# merge new columns with the shots dataframe
shots = pd.merge(left=shots, right=pct_action, how='left', on=["PLAYER1_NAME",	"Year",	"PREVIOUS_DEF_REBOUND",	"PREVIOUS_OFF_MISSED",	"PREVIOUS_OFF_REBOUND"])
shots = pd.merge(left=shots, right=pct_area, how='left', on=["PLAYER1_NAME", "Year", "Shot Zone Basic_Above the Break 3", 
                                                     "Shot Zone Basic_Backcourt", "Shot Zone Basic_In The Paint (Non-RA)", 
                                                     "Shot Zone Basic_Left Corner 3", "Shot Zone Basic_Mid-Range", 
                                                     "Shot Zone Basic_Restricted Area", "Shot Zone Basic_Right Corner 3"])

shots.head()

,Shot Distance,X Location,Y Location,Away Team,Season Type,Year,Shot Zone Basic_Above the Break 3,Shot Zone Basic_Backcourt,Shot Zone Basic_In The Paint (Non-RA),Shot Zone Basic_Left Corner 3,Shot Zone Basic_Mid-Range,Shot Zone Basic_Restricted Area,Shot Zone Basic_Right Corner 3,Shot Zone Area_Back Court(BC),Shot Zone Area_Center(C),Shot Zone Area_Left Side Center(LC),Shot Zone Area_Left Side(L),Shot Zone Area_Right Side Center(RC),Shot Zone Area_Right Side(R),Shot Zone Range_16-24 ft.,Shot Zone Range_24+ ft.,Shot Zone Range_8-16 ft.,Shot Zone Range_Back Court Shot,Shot Zone Range_Less Than 8 ft.,GAME_ID,EVENTNUM,target,PERIOD,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,OPPONENT_TEAM,at_home,PREVIOUS_OFF_REBOUND,PREVIOUS_DEF_REBOUND,PREVIOUS_OFF_TURNOVER,PREVIOUS_OFF_MISSED,PREVIOUS_EVENTMSGTYPE,3PT,jump_shot,layup_shot,dunk_shot,hook_shot,free_throw,DETAILLED_SHOT_TYPE,minutes_left,seconds_left,Age,TS%,PTM,ASTM,ORBM,STLM,BLKM,TOVM,USG%,FG%,2P%,3P%,FT%,PTS,year_start,year_end,height,weight,C,PF,PG,PG-SG,SF,SF-SG,SG,SG-PG,E_DEF_RATING,E_OFF_RATING,PCT_PREV_ACTION,PCT_AREA
0,19.0,-107.0,167.0,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,20000001,11,0,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FREE_THROW,0.0,1.0,0.0,0.0,0.0,0.0,JUMP SHOT,11,668,25.0,0.518,31.084507,4.577465,0.704225,2.507042,0.28169,3.338028,35.9,0.42,0.441,0.32,0.814,2207.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.5,100.7,NaN,35.089974
1,21.0,-115.0,177.0,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,20000001,32,1,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FIELD_GOAL_MADE,0.0,1.0,0.0,0.0,0.0,0.0,JUMP SHOT,7,477,25.0,0.518,31.084507,4.577465,0.704225,2.507042,0.28169,3.338028,35.9,0.42,0.441,0.32,0.814,2207.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.5,100.7,NaN,35.089974
2,16.0,165.0,13.0,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,20000001,34,0,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FIELD_GOAL_MADE,0.0,1.0,0.0,0.0,0.0,0.0,JUMP SHOT,7,443,25.0,0.518,31.084507,4.577465,0.704225,2.507042,0.28169,3.338028,35.9,0.42,0.441,0.32,0.814,2207.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.5,100.7,NaN,35.089974
3,15.0,0.0,150.0,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,20000001,39,1,1,Allen Iverson,PHI,NYK,0.0,False,False,False,False,FOUL,0.0,0.0,0.0,0.0,0.0,1.0,FREE THROW,7,432,25.0,0.518,31.084507,4.577465,0.704225,2.507042,0.28169,3.338028,35.9,0.42,0.441,0.32,0.814,2207.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.5,100.7,NaN,35.089974
4,15.0,0.0,150.0,PHI,0.0,2000.0,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,20000001,40,1,1,Allen Iverson,PHI,NYK,0.0,False,False,False,True,FREE_THROW,0.0,0.0,0.0,0.0,0.0,1.0,FREE THROW,7,432,25.0,0.518,31.084507,4.577465,0.704225,2.507042,0.28169,3.338028,35.9,0.42,0.441,0.32,0.814,2207.0,1997.0,2010.0,182.88,165.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.5,100.7,76.724138,35.089974


In [29]:
# pourcentage of free throws made per player
free_throws = shots[shots.free_throw==True]
pct_free_throw = free_throws.groupby(['PLAYER1_NAME', 'Year']).agg({"target":"mean"}).reset_index()
pct_free_throw.rename({"target":"PCT_FREE_THROW"}, axis=1, inplace=True)

# merge data to the final DF
shots = pd.merge(left=shots, right=pct_free_throw, how='left', on=["PLAYER1_NAME", "Year"])

# replace the PCT_AREA with PCT FREE THROW and drop column
shots.loc[shots.free_throw==True, 'PCT_AREA'] = shots.loc[shots.free_throw==True, 'PCT_FREE_THROW']
shots.drop("PCT_FREE_THROW", axis=1, inplace=True)

# replace NA in PCT_PREV_ACTION with the mean
shots.PCT_PREV_ACTION.fillna(shots.PCT_PREV_ACTION.mean(), inplace=True)

C:\Users\Sarah\AppData\Local\Temp\ipykernel_3856\282025056.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  shots.PCT_PREV_ACTION.fillna(shots.PCT_PREV_ACTION.mean(), inplace=True)


## Add years of experience

In [30]:
shots['YEARS_EXP'] = shots['Year']-shots['year_start']

In [31]:
shots.to_csv('../data/processed/all_shots-v6.csv')